# Cartpole
This code is based on [the cartpole example](https://github.com/stack-of-tasks/pinocchio/blob/master/examples/simulation-pendulum.py) provided by the [Pinocchio](https://stack-of-tasks.github.io/pinocchio/) library. The goal is to balance a pendulum in the upright position by moving the cart below.

The state of the system $x=[y\:\:\:\theta\:\:\:\dot{y}\:\:\:\dot{\theta}]$ is by the position of the cart $y$, the angle of the pendulum $\theta$, and their temporal derivatives. The control input $u$ is the force applied to the cart.

In [1]:
import math
import sys
import time

import hppfcl as fcl
import numpy as np
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer as Visualizer
import ilqr

In [2]:
N = 1  # number of pendulums

model = pin.Model()
geom_model = pin.GeometryModel()

parent_id = 0

cart_radius = 0.1
cart_length = 5 * cart_radius
cart_mass = 2.0
joint_name = "joint_cart"

geometry_placement = pin.SE3.Identity()
geometry_placement.rotation = pin.Quaternion(
    np.array([0.0, 0.0, 1.0]), np.array([0.0, 1.0, 0.0])
).toRotationMatrix()

joint_id = model.addJoint(
    parent_id, pin.JointModelPY(), pin.SE3.Identity(), joint_name
)

body_inertia = pin.Inertia.FromCylinder(cart_mass, cart_radius, cart_length)
body_placement = geometry_placement
# We need to rotate the inertia as it is expressed in the LOCAL frame of the
# geometry.
model.appendBodyToJoint(joint_id, body_inertia, body_placement)

shape_cart = fcl.Cylinder(cart_radius, cart_length)

geom_cart = pin.GeometryObject(
    "shape_cart", joint_id, geometry_placement, shape_cart
)
geom_cart.meshColor = np.array([1.0, 0.1, 0.1, 1.0])
geom_model.addGeometryObject(geom_cart)

parent_id = joint_id

joint_placement = pin.SE3.Identity()
body_mass = 1.0
body_radius = 0.1

for k in range(N):
    joint_name = "joint_" + str(k + 1)
    joint_id = model.addJoint(
        parent_id, pin.JointModelRX(), joint_placement, joint_name
    )

    body_inertia = pin.Inertia.FromSphere(body_mass, body_radius)
    body_placement = joint_placement.copy()
    body_placement.translation[2] = 1.0
    model.appendBodyToJoint(joint_id, body_inertia, body_placement)

    geom1_name = "ball_" + str(k + 1)
    shape1 = fcl.Sphere(body_radius)
    geom1_obj = pin.GeometryObject(geom1_name, joint_id, body_placement, shape1)
    geom1_obj.meshColor = np.ones(4)
    geom_model.addGeometryObject(geom1_obj)

    geom2_name = "bar_" + str(k + 1)
    shape2 = fcl.Cylinder(body_radius / 4.0, body_placement.translation[2])
    shape2_placement = body_placement.copy()
    shape2_placement.translation[2] /= 2.0

    geom2_obj = pin.GeometryObject(geom2_name, joint_id, shape2_placement, shape2)
    geom2_obj.meshColor = np.array([0.0, 0.0, 0.0, 1.0])
    geom_model.addGeometryObject(geom2_obj)

    parent_id = joint_id
    joint_placement = body_placement.copy()

In [3]:
visual_model = geom_model
# Initialize the viewer.
try:
    viz = Visualizer(model, geom_model, visual_model)
    viz.initViewer()
except ImportError as err:
    print(
        "Error while initializing the viewer. "
        "It seems you should install gepetto-viewer"
    )
    print(err)
    sys.exit(0)

try:
    viz.loadViewerModel("pinocchio")
except AttributeError as err:
    print(
        "Error while loading the viewer model. "
        "It seems you should start gepetto-viewer"
    )
    print(err)
    sys.exit(0)

# Display a robot configuration.
q0 = pin.neutral(model)
viz.display(q0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7029/static/


In [14]:
viz.display(np.array([0,1]))

In [25]:
dt = 0.01
T = 5
N = math.floor(T / dt)

model.lowerPositionLimit.fill(-math.pi)
model.upperPositionLimit.fill(+math.pi)
model.lowerPositionLimit[0] = model.upperPositionLimit[0] = 0.0

data_sim = model.createData()

q = pin.randomConfiguration(model)
v = np.zeros(model.nv)

def dynamics(x, u):
    q, v = x[:model.nq], x[model.nq:]

    # apply the ABA algorithm to compute the acceleration
    a = pin.aba(model, data_sim, q, v, u)

    # integrate the acceleration to get the new state
    v += a * dt
    q = pin.integrate(model, q, v * dt)

    return np.concat((q, v))

state_dim = model.nq + model.nv
control_dim = model.nq

Q = np.zeros((state_dim, state_dim)) # state cost
Qf = np.diag([1, 1e5, 1e-1, 2e-1]) # final state cost
R = 1e-5 * np.eye(control_dim) # control cost (minimize the energy)

s = ilqr.ILQRSolver(state_dim, control_dim, Q, Qf, R)

target = np.zeros(state_dim) # target state, upright pendulum with no velocity
controls = s.solve(np.concatenate((q, v)), target, dynamics, time_steps=N, max_iterations=100, initialization=1.0, gradient_clip=100.0)

t = 0.0
for k in range(N):
    tic = time.time()

    tau_control = controls[k]
    x = dynamics(np.concat((q, v)), tau_control)
    q, v = x[:model.nq], x[model.nq:]

    viz.display(q)

    toc = time.time()
    ellapsed = toc - tic
    dt_sleep = max(0, dt - (ellapsed))

    time.sleep(dt_sleep)
    t += dt